In [1]:
from openai import OpenAI
import os
client = OpenAI(
    api_key=''
)

In [7]:
import json
import ast
# read training files

with open('training_data/train_lf.json', 'r', encoding='utf-8') as f:
    content = json.load(f)

tot_prompt = "Imagine five different experts answering this question. All experts will write down 1 step of their thinking and then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes they're wrong at any point, then they leave. And show me the result in the end. \nThe question is:\n\nYou are a recommender system. Given the historical data and some details, analyze the user's interests and try to recommend artists the user might like. Your task is to add at least ten artists between the historical data and the target artist to connect them as an influence path. Then, you will recommend them to the user one by one to influence the user to become interested in the target artist. Any adjacent artists should have a strong relation with each other and make sure the artists are not included in the historical data.\n"

cot_prompt = "You are a recommender system. Given the historical data and some details, analyze the user's interests and try to recommend artists the user might like. Your task is to add at least ten artists between the historical data and the target artist to connect them as an influence path. Then, you will recommend them to the user one by one to influence the user to become interested in the target artist. Any adjacent artists should have a strong relation with each other and make sure the artists are not included in the historical data. Think step by step and make sure.\n"

plain_prompt = "You are a recommender system. Given the historical data and some details, analyze the user's interests and try to recommend artists the user might like. Your task is to add at least ten artists between the historical data and the target artist to connect them as an influence path. Then, you will recommend them to the user one by one to influence the user to become interested in the target artist. Any adjacent artists should have a strong relation with each other and make sure the artists are not included in the historical data.\n"

# suf_lst = ['','_CoT','_ToT']
suf_lst = ['_ToT']
# prompt_lst = [plain_prompt, cot_prompt, tot_prompt]
prompt_lst = [tot_prompt]
    
count = 0
for _suf, _prompt in zip(suf_lst, prompt_lst):
    print(_suf)
    with open(f"training_data/output_gpt_lf{_suf}_pro.json", "w", encoding='utf-8') as fj:
        
        result = []
        for target, his_tra in content["llm"]:
            count+=1
            print(count)
            
            for i in range(2):
                prompt = f"Historical data: {his_tra}\nTarget artist: {target}"
                # print(prompt)
                message = [
                    {"role": "system", "content": _prompt},
                    {"role": "user", "content": prompt}
                ]
                completion = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=message
                )
                
                first_reply = completion.choices[0].message.content
                message.append({"role": "assistant", "content": first_reply})
                message.append({"role": "user", "content": "Output the influence path in the format of python list object. Example output:\n[\"artist1 name\",\n\"artist2 name\",\n\"artist3 name\",\n\"artist4 name\",\n...\n\"target artist name\"]\n"})
                completion = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=message
                )

                influence_path = completion.choices[0].message.content
                
                try:
                    influence_path_get = influence_path[influence_path.index("["):influence_path.index("]")+1]
                    influence_path_lst = ast.literal_eval(influence_path_get)
                except:
                    print(f"GPT输出不合规: First reply: {first_reply}, Second: {influence_path}")
                    influence_path_lst = []
                
                # print(influence_path_get)
                if i == 0:
                    result.append([influence_path_lst])
                elif i == 1:
                    result[-1].append(influence_path_lst)
            # break
        json.dump(result, fj)
    # break

_ToT
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
GPT输出不合规: First reply: Step 1:
From the historical data, I see that the user enjoys pop and rock music, with a preference for female vocalists. I will look for artists who share similar characteristics with the target artist, Zloty Dawai, to create a smooth path of influence., Second: Step 2:
I will start by finding a connection between Zloty Dawai and an artist from the historical data set. This will help me establish a relevant path of influence for the user. Let's see if there are any commonalities in terms of music genre or vocal style between Zloty Dawai and the historical artists.
32
33
34
GPT输出不合规: First reply: Expert 1:

Step 1: Identify artists similar to mujuice + dzhem based on genre and style.

Step 2: Find artists who have collaborated or performed together with mujuice + dzhem.

Step 3: Look for artists who have been influenced by mujuice + dzhem's work.

Step 4: Analyze the historica

In [18]:
import json
import ast
# read training files
prompts = []
with open('training_data/train_1m', 'r', encoding='utf-8') as f:
    content = f.readlines()
    prompt = ""
    for line in content:
        if line == '<end>\n':
            prompts.append(prompt)
            prompt = ""
            continue
        prompt += line
        
SUF = ['','_CoT','_ToT']
_suf = SUF[2]

tot_prompt = "Imagine five different experts answering this question. All experts will write down 1 step of their thinking and then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes they're wrong at any point, then they leave. And show me the result in the end. \nThe question is:\n\nYou are a recommender system. Given the user profile and historical data, analyze the user's interests and try to recommend movies the user might like. Your task is to add at least ten movies between the historical data and the target movie to connect them as an influence path. Then recommend them to the user one by one. Any adjacent movies should have a strong relation with each other, and make sure the movies are not included in the historical data. The movies should be before 2001.\n"

cot_prompt = "You are a recommender system. Given the user profile and historical data, analyze the user's interests and try to recommend movies the user might like. Your task is to add at least ten movies between the historical data and the target movie to connect them as an influence path. Then recommend them to the user one by one. Any adjacent movies should have a strong relation with each other, and make sure the movies are not included in the historical data. Think step by step and make sure. The movies should be before 2001.\n"

plain_prompt = "You are a recommender system. Given the user profile and historical data, analyze the user's interests and try to recommend movies the user might like. Your task is to add at least ten movies between the historical data and the target movie to connect them as an influence path. Then recommend them to the user one by one. Any adjacent movies should have a strong relation with each other, and make sure the movies are not included in the historical data. The movies should be before 2001.\n"

j = 0
with open(f"training_data/output_gpt_1m{_suf}_pro", "w", encoding='utf-8') as f:
    with open(f"training_data/output_gpt_1m{_suf}_pro.json", "w", encoding='utf-8') as fj:
        result = []
        
        for prompt in prompts:
            j+=1
            print(j)
            for i in range(2):
                message = [
                    {"role": "system", "content": cot_prompt},
                    {"role": "user", "content": prompt}
                ]
                completion = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=message
                )
                
                message.append({"role": "assistant", "content": completion.choices[0].message.content})
                message.append({"role": "user", "content": "Output the influence path in the format of python list object. Example output:\n[movie name1,\nmovie name2,\nmovie name3,\nmovie name4,\n...\ntarget movie name]\n"})
                completion = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=message
                )
                
                
                f.write(prompt)
                f.write("\nInfluence path:\n")
                influence_path = completion.choices[0].message.content
                
                
                try:
                    influence_path_get = influence_path[influence_path.index("["):influence_path.index("]")+1]
                    influence_path_lst = ast.literal_eval(influence_path_get)
                except:
                    print(f"GPT输出不合规:{influence_path}")
                    influence_path_lst = []
                
                for influence in influence_path_lst:
                    f.write(f"{influence}\n")
                f.write("<end>\n")
                
                if i == 0:
                    result.append([influence_path_lst])
                elif i == 1:
                    result[-1].append(influence_path_lst)
        json.dump(result, fj)
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123


In [35]:
import json
from transformers import BertModel, BertTokenizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import heapq

with open("movie_name.json", "r") as f:
    movie_name = json.load(f)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
movie_embed = [np.array(tokenizer.encode(movie[1], padding='max_length', max_length=128)).reshape(1, -1) for movie in movie_name]

def find_rel_movie(item, movie_list):
    item_embed = np.array(tokenizer.encode(item, padding='max_length', max_length=128)).reshape(1, -1)
    movie_cosine = [cosine_similarity(item_embed, _) for _ in movie_list]
    
    topk_index = heapq.nlargest(10, range(len(movie_cosine)), movie_cosine.__getitem__)
    
    return topk_index
        
influence_path = completion.choices[0].message.content.split(', ')
# print(movie_name[find_rel_movie('Titanic (1997)', movie_name)])
# for item in influence_path:
#     print(movie_name[find_rel_movie(item, movie_embed)][1])

for index in find_rel_movie('The English Patient (1996)', movie_embed):
    print(movie_name[index])

[1183, 'English Patient, The (1996)']
[2140, 'Dark Crystal, The (1982)']
[2375, 'Money Pit, The (1986)']
[2141, 'American Tail, An (1986)']
[308, 'Three Colors: White (1994)']
[306, 'Three Colors: Red (1994)']
[2740, 'Kindred, The (1986)']
[245, 'Glass Shield, The (1994)']
[307, 'Three Colors: Blue (1993)']
[3200, 'Last Detail, The (1973)']


In [5]:
from openai import OpenAI
import os
client = OpenAI(
    api_key=')

message = [{'role': 'user', 'content': 'Solve a recommendation task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation and current user interest, and \nAction can be the following types: \n(1) recommend[item], which recommend an item to user based on user\'s interest. Your goal is to meet the user\'s interest as much as possible and make recommendations to users as many times as possible. Note that if the user is not satisfied with your recommendations, he will quit and not accept new recommendations\n\nYou may take as many steps as necessary.\nHere are some examples:\n\nQuestion: The user\'s viewing history is [\'Pretty in Pink\', "One Flew Over the Cuckoo\'s Nest", \'Ransom\', \'Saving Private Ryan\', \'X-Men\', \'Coyote Ugly\', \'The Patriot\', \'Me, Myself and Irene\', \'Gone in 60 Seconds\', \'The Perfect Storm\', \'Titanic\', \'The Haunting\', \'Bedknobs and Broomsticks\', \'Clerks\', \'The Matrix\', \'The Shawshank Redemption\', \'Vacation\', \'Father of the Bride\', \'Wallace & Gromit: The Best of Aardman Animation\', \'Back to the Future\', \'Fight Club\'], please recommend item for this user\n\nThought: The user seems to enjoy a mix of drama, action, and comedy. They also seem to appreciate thriller and war films. I would first recommend the user his favorite drama genre movies, and then recommend some other niche genre movies that he likes.\n\nAction: recommend[The Godfather]\n\nObservation: Episode continue, reward=0.90697035862193367\n\nThought:The user seems to have responded positively to this drama and action type film. Based on previous plan, He likes comedy movies, too. I should give him another movie combining comedy and drama.\n\nAction: recommend[The American President]\n\nObservation: Episode continue, reward=0.81253526485664527\n\nThought: It seems that the users are satisfied with the recommendation. Following previous plan, after recommending his favorite genre of movies, we should recommend some other niche genre movies that he likes, such as thriller and war films.\n\nAction: recommend[The Transformers: The Movie]\n\nObservation: Episode continue, reward=0.22653369526521027\n\nThought: This recommendation seems not satisfy the user. This may because the movie was aimed at children, and the user didn\'t like it. In order to retain the user in the long term, we need to recommend his favorite drama movies.\n\nAction: recommend[Braveheart]\n\nObservation: Episode continue, reward=0.96525663552102725\n\nThought: The user like this recommendation. In next step, the new plan should involve considering the user\'s specific preferences and recommending a diverse range of movies across different genres.\n\nAction: recommend[First Knight]\n\nObservation: Episode finished, User Stop, reward=-1000.00000\n\n\n\nQuestion: The user\'s viewing history is [\'Blockland\', \'Shovel Knight: Treasure Trove\', \'PlanetSide 2\', \'Eldritch\', "Shantae: Risky\'s Revenge - Director\'s Cut"], please recommend item for this user\nThought: The user seems to enjoy a mix of Action and Independent video game. They also seem to appreciate Adventure games. I would first recommend the user his favorite action games, and then recommend some other niche genre games that he likes.\n\nAction: recommend[Plain Sight]\n\nObservation: Episode continue, reward=4\n\nThought: The user seems to have responded positively to this Action and Indid type games. Based on previous plan, He likes Adventure games, too. I should recommend him another Adventure game.\n\nAction: recommend[The Path]\n\nObservation: Episode continue, reward=5\n\nThought: It seems that the users are satisfied with the recommendation. Following previous plan, after recommending his favorite genre of games, we should recommend some other niche genre games that he likes, such as RPG games.\n\nAction: recommend[I, Gladiator]\n\nObservation: Episode continue, reward=4\n\nThought: The user like this recommendation. In next step, the new plan should involve considering the user\'s specific preferences and recommending a diverse range of games across different genres.\n\nAction: recommend[Thirty Flights of Loving]\n\nObservation: Episode finished, User Stop, reward=-1000.00000\n\n\n\n\n(END OF EXAMPLES)\n\nYou have attempted to recommend items to users before. The following reflection(s) give some insights on how to better interact with users. Use them to improve your strategy of recommending.\nReflections:\n- The user became dissatisfied with the recommendations because they were not able to receive their preferred game, "Firewatch", and instead received alternative recommendations. The user may have been looking for a specific game or had a strong preference for "Firewatch". To mitigate this failure, the agent should prioritize recommending the user\'s preferred games and ensure that alternative recommendations closely align with the user\'s preferences.\n- The user has become dissatisfied with the recommendations because they have already played or are familiar with the recommended games. Additionally, the user may be looking for more variety in terms of genres and gameplay mechanics. In the future, I should consider the user\'s previous gaming history to avoid recommending games they have already played or are not interested in. I should also provide a diverse range of options by recommending games from different genres and with different gameplay mechanics. This will help to ensure that the user remains engaged and satisfied with the recommendations.\n\nQuestion: The user\'s viewing history is [\'LIMBO\', \'Batman™: Arkham Origins\', \'FEIST\', "Assassin\'s Creed® Unity"], please recommend item for this user\nThought 1:'}]

with open("xxxxy.txt", 'w', encoding='utf-8') as f:
    f.write(message[0]['content'])

completion = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=message,
                    temperature=0.5,
                    max_tokens=4096,
                    top_p=1,
                    stop='\n',
                )

reply = completion.choices[0].message.content
reply

'The user seems to enjoy a mix of action-adventure and puzzle games. They also seem to appreciate games with a dark and mysterious atmosphere. I should prioritize recommending games that align with these preferences.'

In [7]:
# test steam dataset
_ = [4684, 1578, 128, 5409, 4361, 4978, 4869, 5248, 3350, 4389]
tar_list = [137838, 18336, 1392, 173287, 125150, 149903, 145637, 164941, 63989, 125989]

import json
import ast
# read training files
prompts = []
with open('dataset/test_ran.json', 'r', encoding='utf-8') as f:
    content = json.load(f)

for i in range(len(content)):
    if content[i]['userid_encoded'] in tar_list:
        his_seq = content[i]["his_seq"]
        his_msg = ""
        for item in his_seq:
            his_msg += f"{item}\n"
        tar_item = content[i]["target_item"]
        prompts.append([f"Historical data:\n{his_msg}\nTarget item:\n{tar_item}", his_seq, tar_item])

# tot_prompt = "Imagine five different experts answering this question. All experts will write down 1 step of their thinking and then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes they're wrong at any point, then they leave. And show me the result in the end. \nThe question is:\n\nYou are a recommender system. Given the user profile and historical data, analyze the user's interests and try to recommend movies the user might like. Your task is to add at least ten movies between the historical data and the target movie to connect them as an influence path. Then recommend them to the user one by one. Any adjacent movies should have a strong relation with each other, and make sure the movies are not included in the historical data. The movies should be before 2001.\n"

cot_prompt = "You are a recommender system. Given the user profile and historical data, analyze the user's interests and try to recommend games the user might like. Your task is to add at least ten games between the historical data and the target item to connect them as an influence path. Then recommend them to the user one by one. Any adjacent games should have a strong relation with each other, and make sure the games are not included in the historical data. Think step by step and make sure. The games should be before 2018.\n"

# plain_prompt = "You are a recommender system. Given the user profile and historical data, analyze the user's interests and try to recommend movies the user might like. Your task is to add at least ten movies between the historical data and the target movie to connect them as an influence path. Then recommend them to the user one by one. Any adjacent movies should have a strong relation with each other, and make sure the movies are not included in the historical data. The movies should be before 2001.\n"

j = 0
with open(f"training_data/steam.txt", "w", encoding='utf-8') as f:
    with open(f"training_data/steam.json", "w", encoding='utf-8') as fj:
        result = []
        
        for prompt_e in prompts:
            
            prompt = prompt_e[0]
            
            j+=1
            print(j)
            print(prompt)
            for i in range(1):
                message = [
                    {"role": "system", "content": cot_prompt},
                    {"role": "user", "content": prompt}
                ]
                completion = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=message
                )
                
                message.append({"role": "assistant", "content": completion.choices[0].message.content})
                message.append({"role": "user", "content": "Output the influence path in the format of python list object. Example output:\n[game name1,\ngame name2,\ngame name3,\ngame name4,\n...\ntarget item name]\n"})
                completion = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=message
                )
                
                
                f.write(prompt)
                f.write("\nInfluence path:\n")
                influence_path = completion.choices[0].message.content
                
                
                try:
                    influence_path = influence_path.replace("\n", "")
                    influence_path_get = influence_path[influence_path.index("["):influence_path.index("]")+1]
                    influence_path_lst = ast.literal_eval(influence_path_get)
                except:
                    print(f"GPT输出不合规:{influence_path}")
                    influence_path_lst = []
                
                for influence in influence_path_lst:
                    f.write(f"{influence}\n")
                f.write("<end>\n")
                
                if i == 0:
                    result.append([influence_path_lst, prompt_e[1], prompt_e[2]])
                elif i == 1:
                    result[-1].append(influence_path_lst)
        json.dump(result, fj)

1
Historical data:
Shadow Warrior Classic Redux
Riff Racer - Race Your Music!
Outland
Binary Domain
Toren
Papo &amp; Yo
NightSky
L.A. Noire
Submerged
ENSLAVED™: Odyssey to the West™ Premium Edition
STAR WARS™: The Force Unleashed™ II
Beyond Eyes
SPINTIRES™
Homeworld Remastered Collection
Far Cry® 4

Target item:
12 Labours of Hercules II: The Cretan Bull
2
Historical data:
PlanetSide 2
Street Fighter V
How to Survive 2
Team Fortress 2
FINAL FANTASY® XIII
Team Fortress 2
The Witcher 2: Assassins of Kings Enhanced Edition
Team Fortress 2
DRAGON BALL XENOVERSE
Team Fortress 2
Rust
Dead Rising 3 Apocalypse Edition
Fallout 4
Secrets of Grindea
Dirty Bomb®

Target item:
Project Freedom
3
Historical data:
Mount &amp; Blade: Warband
Supreme Commander: Forged Alliance
The Crew™
Super Win the Game
DayZ
Robocraft
Primordia
Shardlight
Robocraft
Lambda Wars Beta
The Binding of Isaac: Rebirth
STAR WARS™ Republic Commando™
Lords Of The Fallen™
Nether: Resurrected
Resident Evil 6 / Biohazard 6

Target

In [ ]:
x = ["Hearthstone: Heroes of Warcraft","Child of Light","Hand of Fate","Armello","Thimbleweed Park","Stardew Valley","Oxenfree","The Witness","Pit People","Pyre","Koi-Koi Japan [Hanafuda playing cards]"]
